In [5]:
! cp ../../../data/liver/objects/model6/tracksColDict.txt ../../../data/liver/objects/model9/
#! cp ../../../data/skin/objects/model6/tracksColDict.txt ../../../data/skin/objects/model9/
#! cp ../../../data/germline/objects/model6/tracksColDict.txt ../../../data/germline/objects/model9/

In [1]:
# imports ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import random
import pandas as pd
import numpy as np 
from numpy.random import choice
import collections
from Bio import AlignIO
import pysam 
from datetime import datetime
import gzip
import multiprocessing
import sys 
import json 
import time # for timing the loop 
#home-made modules : 
sys.path.append('/research/projects/hsapiens/mutability/analysis/global/track_data/annotation/') 
import annotation_handling
sys.path.append('/research/projects/hsapiens/mutability/analysis/modules/closest_value/') 
import closest_val


tmp_file_dir ="../../../"

# command line input ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

# tissue = sys.argv[1]
# model_name = sys.argv[2]
# list_of_surrounding_contexts = json.loads(sys.argv[3]) #note if you chnagee/ increase this, then you oncrese the buffer zone (not using sites in the buffer, len(dna)-max_distance )
# if "_allTissueSpecTracks" in sys.argv: allTissueSpecTracks = True
# else: allTissueSpecTracks = False 

tissue = "liver"
model_name = "model9" 
list_of_surrounding_contexts = [0,100,10000]
allTissueSpecTracks = False

model_desc=""

if tissue == "germline": 
    mutations_lines = open(tmp_file_dir+'data/germline/mutation_data/mutations_hg18_final.bed').readlines()
    mutations_df = pd.read_table(tmp_file_dir+'data/germline/mutation_data/mutations_hg18_final.bed',sep="\t",header = None)
    mutations_df.columns = ["chromosome","start","fake_end","ref","alt","Fathers_age_at_conception","Mothers_age_at_conception"]
elif tissue in ["blood","liver"]:
    mutations_lines = open(tmp_file_dir+'data/{t}/mutations/mutations.bed'.format(t=tissue)).readlines()
    mutations_df = pd.read_table(tmp_file_dir+'data/{t}/mutations/mutations.bed'.format(t=tissue),sep="\t",header = None)
    mutations_df.columns = ["chromosome","start","fake_end","ref","alt","ID","VAF","Gene name", "Region", "AA", "COSMIC", "Species", "Gender", "Age_in_years",           
                            "Tissue/Cell type","Single-cell_genomics_biotechnology_or_Method","Control_sample_or_tissue"]
elif tissue == "skin": 
    mutations_lines = open(tmp_file_dir+'data/skin/mutations/SomaMutDB/mutations_0based.bed'.format(t=tissue)).readlines()
    mutations_df = pd.read_table(tmp_file_dir+'data/skin/mutations/SomaMutDB/mutations_0based.bed'.format(t=tissue),sep="\t",header = None)
    mutations_df.columns = ["chromosome","start","fake_end","ref","alt"]+list(mutations_df.columns)[5:]
else: 
    print(tissue," tissue specified not yet supported  !!~~~!!!~~~~!!")

    

#dictionry where i specify which col contains the information in the datafile , 0 indexed 
tracksColFile_dict = json.load(open(tmp_file_dir+"data/{t}/objects/{m}/tracksColDict.txt".format(m=model_name,t=tissue)))#  




In [3]:


if allTissueSpecTracks == True: 
    model_desc="_allTissueSpecTracks"
    tissue_specific_tracknames = ["H3k27",'H3k4me3',"H3k4me1","H3k27me3","H3k36me3","DNAse","Transcription","methylation"]

    for track in tissue_specific_tracknames: 
        if tissue=="liver" and track=="H3k27me3":  continue
        tracksColFile_dict[track+"_"+tissue] = tracksColFile_dict[track]
        del tracksColFile_dict[track]

    other_tissues = ["blood","skin","liver","germline"]
    other_tissues.remove(tissue)

    for other_tissue in other_tissues: 
        other_tracksColDict = json.load(open(tmp_file_dir+"data/{t}/objects/{m}/tracksColDict.txt".format(m=model_name,t=other_tissue)))
        for track in tissue_specific_tracknames: 
            if not (other_tissue=="liver" and track=="H3k27me3"):
                tracksColFile_dict[track+"_"+other_tissue] = other_tracksColDict[track]
                


#mutant sites ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#create a dictionary where each chrom key will have a n empty list 
duplicate_lines = []
muts_bychrom_dict = {}
for x in range(1,23): 
    key_string = 'chr{n}'.format(n=x)
    muts_bychrom_dict[key_string] = []
    
print(tissue,": fill each chromosome's empty list  with the sites for that chrom")
non_chrnMuts = []#create list of chrom names that dont belong to chrN format --> disgnostic 
for line in (mutations_lines[1:]): 
    if line[0]=="c":                                               #aking sure the line is a chrN (lots of weird junk..) 
        chrom_mut = line.split("\t")[0]
        mut_startSite = line.split("\t")[1]                  #getting rid of the weird double(hgopefully) 
        if chrom_mut in muts_bychrom_dict.keys():                  #controlling for chrX/chrY
            if mut_startSite not in muts_bychrom_dict[chrom_mut]: 
                muts_bychrom_dict[chrom_mut].append(mut_startSite)
            else: duplicate_lines.append(line)
        else: 
            non_chrnMuts.append(chrom_mut)

#testing making usre the only sites that dont make it are sex chromosome mutations 
timestamp = datetime.now().strftime("%Y_%m_%d")
error_log = str("df created on "+timestamp+"\n")
error_log+=(str(len(non_chrnMuts))+"  non chrN muts (ommited) from these lables: "+str(list(np.unique(non_chrnMuts)))+"\n")

#add the sites infro from file 
sites = []#sites = list of sites 
for chrom_key in muts_bychrom_dict.keys(): 
    for mutation_element in muts_bychrom_dict[chrom_key]: 
        sites.append([chrom_key, int(mutation_element),1]) #the 1 is for mutation status column. 1 = yes 


#non -mutant sites ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#get chrom length information so I can perform weighted choice for non-mut site selection"
ChromLengths = pd.read_csv(tmp_file_dir+'data/global/sequence/hg38_chromosomelengths.csv') #read in the csv file of hg38 chrom lengths I found on the internets 
total_length=0 #lets sum (get the total length) 
for length in list(ChromLengths.Length): 
    total_length+=int(length.replace(",",""))

#build dictionary to store porbability 
dict_lengths = {}#creat emepty dictionary 
for x in range (0,22): 
        tmp_index = x +1
        length = str(ChromLengths[x:x+1]).split()[4]
        length = length.replace(",", "")
        length = int(length)
        dict_lengths["chr"+str(tmp_index)] = length

#make the porbability of choosing a chrom based on length 
list_chroms = ['chr' + str(i) for i in range(1, 23)]
list_chrom_probabilities = []
for chrom in list_chroms: 
    list_chrom_probabilities.append(dict_lengths[chrom]/total_length)
list_chrom_probabilities[0] = list_chrom_probabilities[0]+1-sum(list_chrom_probabilities) # adds the 0.00000001 left from rounding errors to the chr1 so sum adds perfectly to 1. 
assert(sum(list_chrom_probabilities)==1)

#perfrom the non-mutant site draw 
number_nonmuts = int(len(sites)*1.2)
chrom_draw = choice(list_chroms, number_nonmuts,p=list_chrom_probabilities)

print(tissue,"make the sites list with the chr# and site" )
for i in (range(1,23)): 
    chrom = "chr"+str(i)
    chrom_nchoose = list(chrom_draw).count("chr"+str(i))
    chrom_sites_chosen = random.sample(range(1, dict_lengths[chrom]), chrom_nchoose) #without duplucates 
    for j in chrom_sites_chosen: 
        sites.append([chrom,j,0])# the 0 if for the mutation status column. 0 = no 
        


liver : fill each chromosome's empty list  with the sites for that chrom
liver make the sites list with the chr# and site


In [4]:

# genral declarations before the big function ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

distance_max = max(list_of_surrounding_contexts)

fastas_dict = {}   # creating dictionary with fasta alignment, length of seq, 
print(tissue,"making the fastas dictionary")
for chrom in (list_chroms):
    filename_tmp = tmp_file_dir+"data/global/sequence/{c}.fa.gz".format(c=chrom)
    fastas_dict[chrom] = []
    with gzip.open(filename_tmp, "rt") as handle:
        fastas_dict[chrom].append(AlignIO.read(handle,"fasta"))
        alignment_tmp = fastas_dict[chrom][0]
        fastas_dict[chrom].append(len(str(alignment_tmp[0].seq)))

                 
#generate header ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~        

header = "Chromosome"+ "\t"+"site"+"\t" +"triplet"+"\t"+"mutation_status"                        # creating the begining of the header 
for trackname in tracksColFile_dict.keys():   # the rest of the header is a function of tracks 
    if trackname in ["annotation","mappability","dist_rep_org_main","dist_rep_org_all","CpGisland"]: 
        header = header + "\t"+str(trackname)
    elif  "methylation" in trackname:
        for distance in list_of_surrounding_contexts: 
            header = header + "\t"+str(trackname)+"_precent"+"-"+str(distance)+ "\t"+str(trackname)+"_coverage"+"-"+str(distance)
    else: 
        for distance in list_of_surrounding_contexts:                                                # and distance (need a col for every track and for every distance value within ) 
            header = header + "\t"+str(trackname)+"-"+str(distance)
for distance in list_of_surrounding_contexts:                                                    # creating the end of the header assoc with no track (the seqeunce at different 
    header = header + "\t"+"Apercent-"+str(distance)+ "\t"+"Gpercent-"+str(distance)+ "\t"+"Cpercent-"+str(distance)+ "\t"+"Tpercent-"+str(distance)   # distace values) 
header = header +"\n"                                                                            # obviously needs to end with a \n 


filename_df = tmp_file_dir+'data/{a}/dataframes/{m}/predictorDf{d}.txt'.format(a=tissue,m=model_name,d=model_desc)
with open(filename_df,"w") as f: 
    f.write(header)     
    

liver making the fastas dictionary


In [5]:

    
#big boy function 
def predictor_rowString(site): 
    row = []
    if site[1] <= distance_max or site[1]+distance_max >= fastas_dict[site[0]][1]:               # only use sites that will have values for site +- max distance (buffer). second element in fastas dict is the length 
        row.extend([str(site),"out of buffer range"])

    else: 
        row.extend([site[0], site[1]])
        alignment = fastas_dict[site[0]][0]                                                       #create the alingment from the list of fastas 

        #makes the triplet and tests alignment 
        if site[2] ==1:                                     #muts_by-chrom_dict is literally a dictionary containing list of sites that are mutations in that chrom. 
            mutation_row = mutations_df[(mutations_df.chromosome == site[0]) & (mutations_df.start == site[1])]  #get the row containing mut info out of the df 
            old_bp = mutation_row.ref.values[0]
            old_triplet = (str(alignment[0,site[1]-1])+str(old_bp)+str(alignment[0,site[1]+1])).upper()
            row.extend([old_triplet, 1])#usin the old/ref triplet in the df instead (the mutation happened to the old triplet! )

            seq_triplet = str(alignment[0,site[1]-1:site[1]+2].seq)
            if old_triplet.upper() != seq_triplet.upper():                                                          #testing that 
                row.append("discordant. triplet using daata = "+old_triplet+", seqeunce triplet = "+seq_triplet)
        else: 
            triplet= str(alignment[0,site[1]-1:site[1]+2].seq).upper()
            row.extend([triplet,0])

        for trackname,track_val in tracksColFile_dict.items():    
            data_col = track_val[0] 
            global_or_tissue_specific = track_val[1]
            Na_is_0_or_NA = track_val[2]
            filename = tmp_file_dir+track_val[3]

            if trackname == "annotation": 
                if not [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1], site[1]+1)]:                #if no value at that site 
                     row.append("not_transcribed")
                else: 
                    track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1], site[1]+1)]
                    old_labels = [element.split()[2] for element in track_output]
                    converted_list, final_label,alien_labels = [],str(),[] 
                    for label in old_labels: 
                        if label in annotation_handling.annotation_conversion.keys():  #even though i though i controlled for it, occasiaonlyl there would eb anew label, so sontrol for this 
                            converted_list.append(annotation_handling.annotation_conversion[label])
                        else: 
                            alien_labels.append(label)
                    final_label = annotation_handling.annotation_priorityLabel(converted_list)
                    if len(alien_labels) != 0:    # if there is an "alien" annotation label, then just add that into the position and i can handle on ind basis later 
                        for label in alien_labels: 
                            final_label+="_"+label
                    row.append(final_label)
            elif trackname == "mappability": 
                if [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]: row.append("mappable")
                else: row.append("not")
            elif trackname in ["dist_rep_org_main","dist_rep_org_all"]: 
                row.append(closest_val.shortest_distance(site,filename))
            elif trackname == "CpGisland": 
                if [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1], site[1]+1)]: row.append("island")
                elif closest_val.shortest_distance(site,filename) <= 2000: row.append("shore")
                else: row.append("not")
            elif "methylation" in trackname:
                #########################################working below 
                for distance in list_of_surrounding_contexts: 
                    track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+1+distance)]
                    if track_output: 
                        num_reads_list,percent_methyl_list,next_door = [],[],False
                        for record in track_output: 
                            cur_site = int(record.split("\t")[1])
                            if cur_site == site[1]: 
                                num_reads_at_site,percent_methylated = record.split("|")[6:8]
                                if int(num_reads_at_site) >= 3: 
                                    num_reads_list.append(int(num_reads_at_site))
                                    percent_methyl_list.append(float(percent_methylated))
                            elif cur_site in [site[1]-1,site[1]+1] and distance in [0,1]: next_door = True
                        if num_reads_list: # if the list has eleemtns 
                            row.extend([np.mean(percent_methyl_list),np.mean(num_reads_list)])
                        else: # if the list is empty 
                            if next_door==True: #if the only reason the result is empty is that there is nothing htere, but tabix grabbed it b/c next door 
                                row.extend(["no_percent_data","no_coverage_data"])
                            else: #the result is empty because all the returns had coverage less than 3 
                                row.extend(["infsuff_coverage","infsuff_coverage"])
                #         else: 
                #             for record in track_output:
                #                 num_reads_at_site,percent_methylated = record.split("|")[6:8]
                #                 if int(num_reads_at_site) >= 3: 
                #                     num_reads_list.append(int(num_reads_at_site))
                #                     percent_methyl_list.append(float(percent_methylated))
                #             row.extend([np.mean(percent_methyl_list),np.mean(num_reads_list)])
                    else: 
                        row.extend(["no_percent_data","no_coverage_data"])
            ############################################################################working abovce
            else: 
                for distance in list_of_surrounding_contexts: 
                    if not [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]:                #if no value at that site 
                        if Na_is_0_or_NA == "Na=0": 
                            row.append(0)
                        else: 
                            row.extend(["NA"])                                                                                      
                    else:                                                   
                        track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]
                        multiple_values = []

                        if tracksColFile_dict[trackname][0] == 4:
                            for element in track_output: 
                                multiple_values.append(float(element.split()[4]))
                            average_value = sum(multiple_values)/len(multiple_values) 
                            row.append(average_value)
                        elif tracksColFile_dict[trackname][0] == 3: 
                            for element in track_output: 
                                multiple_values.append(float(element.split()[3]))
                            average_value = sum(multiple_values)/len(multiple_values) 
                            row.append(average_value)
                        elif tracksColFile_dict[trackname][0] == 'binary': 
                            row.append(len(track_output))
                        else: 
                            row.append(((str(site)+"ERROR_track_coloumns_not_4_or_5_or_binary_"+trackname)))
                            

        #sequence stuff                
        for distance in list_of_surrounding_contexts: 
            seq_around = str(alignment[0,site[1]-1:site[1]+2].seq)
            if seq_around != '': 
                seq_around = str(alignment[0,site[1]-distance:site[1]+distance+1].seq)
                Acount = seq_around.count('a')+seq_around.count("A")
                Gcount = seq_around.count('g')+seq_around.count("G")
                Ccount = seq_around.count('c')+seq_around.count("C")
                Tcount = seq_around.count('t')+seq_around.count("T")
                Apercent = Acount/len(seq_around)
                Gpercent = Gcount/len(seq_around)
                Cpercent = Ccount/len(seq_around)
                Tpercent = Tcount/len(seq_around)
                row.extend([Apercent, Gpercent, Cpercent, Tpercent])
            else: 
                row.extend(['NA','NA','NA','NA'])
                list_no_seq_at_site.append(site)

    row_string = str()
    for i in range(0,len(row)): 
        row_string = row_string+str(row[i])+"\t"
    row_string = row_string.rstrip("\t") # dont need to add the "\n" here as it is added below int he f.write 
    #     row_string = row_string+"\n" 
    return(row_string)





In [6]:
#WRITE THE MODEL #
filename_df = tmp_file_dir+'data/{a}/dataframes/{m}/predictorDf{d}.txt'.format(a=tissue,m=model_name,d=model_desc)
start_time = time.time()
print(tissue,"starting big loop")

liver starting big loop


In [ ]:

def rowString_handler():
    p = multiprocessing.Pool(10)
    with open(filename_df, 'a') as f:
        for result in p.imap(predictor_rowString, sites):
            f.write('%s\n' % result)

if __name__=='__main__':
    rowString_handler()

error_log += (("creating the df loop took "+str(time.time()-start_time)[0:4]+" seconds\n"))

#writing error log to file 
with open(tmp_file_dir+"data/{a}/dataframes/{m}/predictorDf{d}_errorlog.txt".format(a=tissue,m=model_name,d=model_desc),"w") as f: 
      f.write(error_log)


In [8]:
%%bash 
cd ../../../
grep 'buffer' data/skin/dataframes/model9/predictorDf.txt >> data/skin/dataframes/model9/predictorDf_errorlog.txt
grep 'discord' data/skin/dataframes/model9/predictorDf.txt >> data/skin/dataframes/model9/predictorDf_errorlog.txt
grep -v 'discord' data/skin/dataframes/model9/predictorDf.txt > data/skin/dataframes/model9/predictorDf_noDiscord.txt
grep -v 'buffer' data/skin/dataframes/model9/predictorDf_noDiscord.txt >  data/skin/dataframes/model9/predictorDf.txt

In [176]:
%%bash 
cd ../../../data/global/track_data/annotation/
sort -k1,1 -k4,4n global_annotation.gff > global_annotation_sorted.gff
bgzip global_annotation_sorted.gff
tabix -p gff global_annotation_sorted.gff.gz

In [7]:
%%bash 
cd ../../../
grep 'buffer' data/blood/dataframes/model7/predictorDf_allTissueSpecTracks.txt >> data/blood/dataframes/model7/predictorDf_allTissueSpecTracks_errorlog.txt
grep 'discord' data/blood/dataframes/model7/predictorDf_allTissueSpecTracks.txt >> data/blood/dataframes/model7/predictorDf_allTissueSpecTracks_errorlog.txt
grep -v 'discord' data/blood/dataframes/model7/predictorDf_allTissueSpecTracks.txt > data/blood/dataframes/model7/predictorDf_allTissueSpecTracks_noDiscord.txt
grep -v 'buffer' data/blood/dataframes/model7/predictorDf_allTissueSpecTracks_noDiscord.txt > data/blood/dataframes/model7/predictorDf_allTissueSpecTracks_test.txt

In [5]:
%%bash
grep "buffer" ../../../data/liver/dataframes/model4/predictorDf.txt >> ../../../data/liver/dataframes/model4/predictorDf_discordantBuffer.txt
grep "discord" ../../../data/liver/dataframes/model4/predictorDf.txt >> ../../../data/liver/dataframes/model4/predictorDf_discordantBuffer.txt
grep -v "discord" ../../../data/liver/dataframes/model4/predictorDf.txt >  ../../../data/liver/dataframes/model4/predictorDf_noDiscord.txt
grep -v "buffer" ../../../data/liver/dataframes/model4/predictorDf_noDiscord.txt >  ../../../data/liver/dataframes/model4/predictorDf.txt

# extract the code from the loop and paste here so can trouble shoot directly 

In [13]:

    
#big boy function 
for sit in sites[0:10]: 
    row = []
    if site[1] <= distance_max or site[1]+distance_max >= fastas_dict[site[0]][1]:               # only use sites that will have values for site +- max distance (buffer). second element in fastas dict is the length 
        row.extend([str(site),"out of buffer range"])

    else: 
        row.extend([site[0], site[1]])
        alignment = fastas_dict[site[0]][0]                                                       #create the alingment from the list of fastas 

        #makes the triplet and tests alignment 
        if site[2] ==1:                                     #muts_by-chrom_dict is literally a dictionary containing list of sites that are mutations in that chrom. 
            mutation_row = mutations_df[(mutations_df.chromosome == site[0]) & (mutations_df.start == site[1])]  #get the row containing mut info out of the df 
            old_bp = mutation_row.ref.values[0]
            old_triplet = (str(alignment[0,site[1]-1])+str(old_bp)+str(alignment[0,site[1]+1])).upper()
            row.extend([old_triplet, 1])#usin the old/ref triplet in the df instead (the mutation happened to the old triplet! )

            seq_triplet = str(alignment[0,site[1]-1:site[1]+2].seq)
            if old_triplet.upper() != seq_triplet.upper():                                                          #testing that 
                row.append("discordant. triplet using daata = "+old_triplet+", seqeunce triplet = "+seq_triplet)
        else: 
            triplet= str(alignment[0,site[1]-1:site[1]+2].seq).upper()
            row.extend([triplet,0])

        for trackname,track_val in tracksColFile_dict.items():    
            data_col = track_val[0] 
            global_or_tissue_specific = track_val[1]
            Na_is_0_or_NA = track_val[2]
            filename = tmp_file_dir+track_val[3]

            if trackname == "annotation": 
                if not [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1], site[1]+1)]:                #if no value at that site 
                     row.append("not_transcribed")
                else: 
                    track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1], site[1]+1)]
                    old_labels = [element.split()[2] for element in track_output]
                    converted_list, final_label,alien_labels = [],str(),[] 
                    for label in old_labels: 
                        if label in annotation_handling.annotation_conversion.keys():  #even though i though i controlled for it, occasiaonlyl there would eb anew label, so sontrol for this 
                            converted_list.append(annotation_handling.annotation_conversion[label])
                        else: 
                            alien_labels.append(label)
                    final_label = annotation_handling.annotation_priorityLabel(converted_list)
                    if len(alien_labels) != 0:    # if there is an "alien" annotation label, then just add that into the position and i can handle on ind basis later 
                        for label in alien_labels: 
                            final_label+="_"+label
                    row.append(final_label)
            elif trackname == "mappability": 
                if [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]: row.append("mappable")
                else: row.append("not")
            elif trackname in ["dist_rep_org_main","dist_rep_org_all"]: 
                row.append(closest_val.shortest_distance(site,filename))
            elif trackname == "CpGisland": 
                if [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1], site[1]+1)]: row.append("island")
                elif closest_val.shortest_distance(site,filename) <= 2000: row.append("shore")
                else: row.append("not")
            elif "methylation" in trackname:
                #########################################working below 
                for distance in list_of_surrounding_contexts: 
                    track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+1+distance)]
                    if track_output: 
                        num_reads_list,percent_methyl_list,next_door = [],[],False
                        for record in track_output: 
                            cur_site = int(record.split("\t")[1])
                            if cur_site == site[1]: 
                                num_reads_at_site,percent_methylated = record.split("|")[6:8]
                                if int(num_reads_at_site) >= 3: 
                                    num_reads_list.append(int(num_reads_at_site))
                                    percent_methyl_list.append(float(percent_methylated))
                            elif cur_site in [site[1]-1,site[1]+1] and distance in [0,1]: next_door = True
                        if num_reads_list: # if the list has eleemtns 
                            row.extend([np.mean(percent_methyl_list),np.mean(num_reads_list)])
                        else: # if the list is empty 
                            if next_door==True: #if the only reason the result is empty is that there is nothing htere, but tabix grabbed it b/c next door 
                                row.extend(["no_percent_data","no_coverage_data"])
                            else: #the result is empty because all the returns had coverage less than 3 
                                row.extend(["infsuff_coverage","infsuff_coverage"])
                #         else: 
                #             for record in track_output:
                #                 num_reads_at_site,percent_methylated = record.split("|")[6:8]
                #                 if int(num_reads_at_site) >= 3: 
                #                     num_reads_list.append(int(num_reads_at_site))
                #                     percent_methyl_list.append(float(percent_methylated))
                #             row.extend([np.mean(percent_methyl_list),np.mean(num_reads_list)])
                    else: 
                        row.extend(["no_percent_data","no_coverage_data"])
            ############################################################################working abovce
            else: 
                for distance in list_of_surrounding_contexts: 
                    if not [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]:                #if no value at that site 
                        if Na_is_0_or_NA == "Na=0": 
                            row.append(0)
                        else: 
                            row.extend(["NA"])                                                                                      
                    else:                                                   
                        track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+distance+1)]
                        multiple_values = []

                        if tracksColFile_dict[trackname][0] == 4:
                            for element in track_output: 
                                multiple_values.append(float(element.split()[4]))
                            average_value = sum(multiple_values)/len(multiple_values) 
                            row.append(average_value)
                        elif tracksColFile_dict[trackname][0] == 3: 
                            for element in track_output: 
                                multiple_values.append(float(element.split()[3]))
                            average_value = sum(multiple_values)/len(multiple_values) 
                            row.append(average_value)
                        elif tracksColFile_dict[trackname][0] == 'binary': 
                            row.append(len(track_output))
                        else: 
                            row.append(((str(site)+"ERROR_track_coloumns_not_4_or_5_or_binary_"+trackname)))
                            

        #sequence stuff                
        for distance in list_of_surrounding_contexts: 
            seq_around = str(alignment[0,site[1]-1:site[1]+2].seq)
            if seq_around != '': 
                seq_around = str(alignment[0,site[1]-distance:site[1]+distance+1].seq)
                Acount = seq_around.count('a')+seq_around.count("A")
                Gcount = seq_around.count('g')+seq_around.count("G")
                Ccount = seq_around.count('c')+seq_around.count("C")
                Tcount = seq_around.count('t')+seq_around.count("T")
                Apercent = Acount/len(seq_around)
                Gpercent = Gcount/len(seq_around)
                Cpercent = Ccount/len(seq_around)
                Tpercent = Tcount/len(seq_around)
                row.extend([Apercent, Gpercent, Cpercent, Tpercent])
            else: 
                row.extend(['NA','NA','NA','NA'])
                list_no_seq_at_site.append(site)

    row_string = str()
    for i in range(0,len(row)): 
        row_string = row_string+str(row[i])+"\t"
    row_string = row_string.rstrip("\t") # dont need to add the "\n" here as it is added below int he f.write 
    #     row_string = row_string+"\n" 
    print(row_string)





[W::hts_idx_load3] The index file is older than the data file: ../../../data/liver/track_data/H3k27ac/H3k27ac.bed.gz.tbi
[W::hts_idx_load3] The index file is older than the data file: ../../../data/liver/track_data/H3k27ac/H3k27ac.bed.gz.tbi
[W::hts_idx_load3] The index file is older than the data file: ../../../data/liver/track_data/H3k27ac/H3k27ac.bed.gz.tbi
[W::hts_idx_load3] The index file is older than the data file: ../../../data/liver/track_data/H3k27ac/H3k27ac.bed.gz.tbi
[W::hts_idx_load3] The index file is older than the data file: ../../../data/liver/track_data/H3k27ac/H3k27ac.bed.gz.tbi
[W::hts_idx_load3] The index file is older than the data file: ../../../data/liver/track_data/H3k27ac/H3k27ac.bed.gz.tbi


ValueError: could not create iterator for region 'chr1:1843628-1843628'

In [14]:
filename

'../../../data/liver/track_data/H3k27me3/H3k27me3.bed.gz'

In [15]:
!zcat ../../../data/liver/track_data/H3k27me3/H3k27me3.bed.gz | head 

In [22]:
!ls -lah ../../../data/liver/track_data/H3k27me3/

total 7.6G
drwxr-xr-x  3 omanmade domain users 4.0K Jul  9 15:26 .
drwxr-xr-x 12 omanmade domain users  218 Sep 21  2022 ..
-rw-r--r--  1 omanmade domain users    0 Jul  9 15:26 H3k27me3.bed
-rw-r--r--  1 omanmade domain users  11M Jul  9 15:26 H3k27me3_unliftgedhg19.bed
-rw-r--r--  1 omanmade domain users  52M Jul  9 15:17 H3k27me3_unliftgedhg38.bed
-rw-r--r--  1 omanmade domain users 7.5G Jul  9 15:26 H3k27me3_unsorted.bed
-rw-r--r--  1 omanmade domain users 221K Jun  4  2009 hg19ToHg18.over.chain.gz
-rw-r--r--  1 omanmade domain users  948 Jul  9 14:47 hg19ToHg18_wget_output.txt
-rw-r--r--  1 omanmade domain users 1.2M Dec 31  2013 hg38ToHg19.over.chain.gz
-rw-r--r--  1 omanmade domain users 2.5K Jul  9 14:47 hg38ToHg19_wget_output.txt
drwxr-xr-x  2 omanmade domain users    6 Jul  9 14:43 .ipynb_checkpoints
-rw-r--r--  1 omanmade domain users 2.2M Jul  9 14:45 wget_output.txt


# scratch

In [12]:
! awk '{print NF}' ../../../data/liver/dataframes/model9/predictorDf.txt | sort | uniq -c 

  29412 54


In [17]:
header_blood = open("../../../data/blood/dataframes/model9/predictorDf.txt").readlines()[0].split()
header_liver = open("../../../data/liver/dataframes/model9/predictorDf.txt").readlines()[0].split()
header_skin = open("../../../data/skin/dataframes/model9/predictorDf.txt").readlines()[0].split()
header_germline = open("../../../data/germline/dataframes/model9/predictorDf.txt").readlines()[0].split()

In [19]:
len(header_blood), len(header_liver), len(header_germline), len(header_skin)

(51, 54, 57, 60)

In [25]:
for colname in header_germline: 
    if colname not in header_liver: 
        print(colname)

H3k27me3-0
H3k27me3-100
H3k27me3-10000


In [7]:
! awk 'NF==7{print $0}' ../../../data/skin/dataframes/model9/predictorDf.txt | head 

['chr1', 248306415, 0]	out of buffer range
['chr1', 248883641, 0]	out of buffer range
['chr1', 247425269, 0]	out of buffer range
['chr1', 248149009, 0]	out of buffer range
['chr5', 180910052, 0]	out of buffer range
['chr5', 965, 0]	out of buffer range
['chr5', 181494540, 0]	out of buffer range
['chr5', 181063363, 0]	out of buffer range
['chr5', 181528543, 0]	out of buffer range
['chr7', 159206929, 0]	out of buffer range


In [11]:
from tqdm import tqdm
# imports ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import random
import pandas as pd
import numpy as np 
from numpy.random import choice
import collections
from Bio import AlignIO
import pysam 
from datetime import datetime
import gzip
import multiprocessing
import sys 
import json 
import time 

In [12]:
filename_tmp ="../../../"+"data/global/sequence/chr1.fa.gz"
with gzip.open(filename_tmp, "rt") as handle:
    alignment = (AlignIO.read(handle,"fasta"))

sites=[]
tmp_file_dir = "../../../"
#get chrom length information so I can perform weighted choice for non-mut site selection"
ChromLengths = pd.read_csv(tmp_file_dir+'data/global/sequence/hg38_chromosomelengths.csv') #read in the csv file of hg38 chrom lengths I found on the internets 
total_length=0 #lets sum (get the total length) 
for length in list(ChromLengths.Length): 
    total_length+=int(length.replace(",",""))

#build dictionary to store porbability 
dict_lengths = {}#creat emepty dictionary 
for x in range (0,22): 
        tmp_index = x +1
        length = str(ChromLengths[x:x+1]).split()[4]
        length = length.replace(",", "")
        length = int(length)
        dict_lengths["chr"+str(tmp_index)] = length

#make the porbability of choosing a chrom based on length 
list_chroms = ['chr' + str(i) for i in range(1, 23)]
list_chrom_probabilities = []
for chrom in list_chroms: 
    list_chrom_probabilities.append(dict_lengths[chrom]/total_length)
list_chrom_probabilities[0] = list_chrom_probabilities[0]+1-sum(list_chrom_probabilities) # adds the 0.00000001 left from rounding errors to the chr1 so sum adds perfectly to 1. 
assert(sum(list_chrom_probabilities)==1)

#perfrom the non-mutant site draw 
number_nonmuts = 10000
chrom_draw = choice(list_chroms, number_nonmuts,p=list_chrom_probabilities)

# print(tissue,"make the sites list with the chr# and site" )
for i in (range(1,23)): 
    chrom = "chr"+str(i)
    chrom_nchoose = list(chrom_draw).count("chr"+str(i))
    chrom_sites_chosen = random.sample(range(1, dict_lengths[chrom]), chrom_nchoose) #without duplucates 
    for j in chrom_sites_chosen: 
        sites.append([chrom,j,0])# the 0 if for the mutation status column. 0 = no 

In [19]:
# site = ['chr1',1212518]
filename="../../../data/germline/track_data/methylation/methylation.bed.gz"
list_of_surrounding_contexts=[0]

list_rows =[]
# for site in (sites[0:880]): 
site = ["chr14",21919600]
row = [site[0],site[1],alignment[0][site[1]].upper() ]

#     track_output =['chr2\t91661791\t91661792\t"0"|1|-|93087629|93087630|0,255,0|3|0|CCT|CCT|1',
#      'chr2\t91661791\t91661792\t"0"|1|-|93740591|93740592|0,255,0|1|100|CCT|CCT|1',
#      'chr2\t91661791\t91661792\t"0"|1|-|94142516|94142517|0,255,0|1|0|CCT|CCT|3',
#      'chr2\t91661791\t91661792\t"0"|4|-|4393|4394|0,255,0|4|0|CCT|CCT|2']

#########################################working below 
for distance in list_of_surrounding_contexts: 
    track_output = [record for record in pysam.Tabixfile(filename).fetch(site[0], site[1]-distance, site[1]+1+distance)]
    if track_output: 
        num_reads_list,percent_methyl_list,next_door = [],[],False
        for record in track_output: 
            cur_site = int(record.split("\t")[1])
            if cur_site == site[1]: 
                num_reads_at_site,percent_methylated = record.split("|")[6:8]
                if int(num_reads_at_site) >= 3: 
                    num_reads_list.append(int(num_reads_at_site))
                    percent_methyl_list.append(float(percent_methylated))
            elif cur_site in [site[1]-1,site[1]+1] and distance in [0,1]: next_door = True
        if num_reads_list: # if the list has eleemtns 
            row.extend([np.mean(percent_methyl_list),np.mean(num_reads_list)])
        else: # if the list is empty 
            if next_door==True: #if the only reason the result is empty is that there is nothing htere, but tabix grabbed it b/c next door 
                row.extend(["no_percent_data","no_coverage_data"])
            else: #the result is empty because all the returns had coverage less than 3 
                row.extend(["infsuff_coverage","infsuff_coverage"])
#         else: 
#             for record in track_output:
#                 num_reads_at_site,percent_methylated = record.split("|")[6:8]
#                 if int(num_reads_at_site) >= 3: 
#                     num_reads_list.append(int(num_reads_at_site))
#                     percent_methyl_list.append(float(percent_methylated))
#             row.extend([np.mean(percent_methyl_list),np.mean(num_reads_list)])
    else: 
        row.extend(["no_percent_data","no_coverage_data"])
list_rows.append(row)

In [20]:
row

['chr14', 21919600, 'T', 'infsuff_coverage', 'infsuff_coverage']

In [14]:
row

['chr1', 3008881, 'A', 'no_percent_data', 'no_coverage_data']